In [1]:
from IPython.display import HTML
from image_functions import *

### <center> 2.2.2 Last two problems

Two issues remain to be corrected. The first one was already presented in section *2.1.4 Limitations of the Angle-Based Algorithm.* It concerned the behavior of the angular algorithm for spiral turns that the vertical line does not intersect. The problem was that the angular algorithm lands the radius vector onto the $x$-axis.

In [2]:
load_image('fig135')

Here, we need to make an important clarification about which radius vectors are being landed onto the $x$-axis. Since we have already added the $\text{ISSCDD}$-check, we will see that it deactivates $\text{AB-Alg}$ in some cases but not in others. In the two images below, we show the first initial radius vector and the final radius vector at the $5$-th intersection point.

In [5]:
display(load_images([('fig136', 'left'), ('fig137', 'right')]))

As can be seen, the derivatives have the same sign (positive slope) at the beginning and at the end of the iterative process, and $\text{ISSCDD}$ gives a "green light" for $\text{AB-Alg}$. However, the situation is different for the intersection points on the positive side of the ordinate axis.

In [6]:
display(load_images([('fig138', 'left'), ('fig139', 'right')]))

In this case, on the fourth iteration ($m=3$), the derivative changes its sign, and $\text{ISSCDD}$ nullifies $\text{AB-Alg}$. These behaviors of the algorithm are due to the position of $\text{PMD}$. In the upper case, the "descending" radius vector passes $\text{PMD}$, causing the derivative to change its sign. In the previous case, $\text{PMD}$ is positioned above the  $x$-axis (i.e., after it in terms of time $t$), which is why the derivative does not change its sign. 

This is a problem for which we will define a check whose role is to prevent the start of $\text{AB-Alg}$ under certain conditions. We will leave this for last. Before that, we will address and solve the other problem shown in the image below.

In [7]:
load_image('fig140')

The vertical line is positioned between $\text{PMD}$ and the intersection point of the spiral with the $x$-axis, and $\text{AB-Alg}$ cannot move the radius vector from any of the initial vectors. In one case, at $n=1$, the radius vector lands on the $x$-axis, while in the other, it reaches the intersection point with the line.

In [8]:
display(load_images([('fig141', 'left'), ('fig142', 'right')]))

This problem is the same as the missing point issue occurring at $n=0$, with the difference that it appears for $n>0$. The solution is the same - using $\text{LB-Alg}$. This means that it should be added as a third term to the equation for the two-dimensional sequence, with the necessary checks as multipliers in front of it.

$$\lim_{n, m \to \infty} \left\{ t_{[n; m]} \right\} =
\text{ISSCDD}_{(x'_s(t_{[n;0]}), x'_s(t_{[n;m]}))}^{[0 \vee 1]}
\left( \overline{NSwitch_{(n)}^{[0 \vee  1]}}
\text{XMD}_{(x_l, y_s(t_{[1; 0]}))}^{[0 \vee 1]}
\text{SCDD}_{(x'_s(0), x'_s(t_{[0;0]}))}^{[0 \vee 1]}\\
\left(
KWL_{(k, w, x_l)}^{[0 \vee  1]} +
KL_{(k, x_l)}^{[0 \vee  1]}
\right)
\text{LB-Alg}
+ NSwitch_{(n)}^{[0 \vee  1]}
\left( \overline {XYSwitch_{(a)}^{[ 0 \vee 1]}} \text{AB-Alg} +
XYSwitch_{(a)}^{[ 0 \vee 1]} \text{LB-Alg}
\right)
\right), \quad n, m \ge 0.  \quad\quad\quad\quad (2.2.2.1)$$

Since this $\text{LB-Alg}$ occurs at $n>0$, we place both algorithms in parentheses before $NSwitch_{(n)}^{[0 \vee  1]}$, with the additional condition $XYSwitch_{(a)}^{[ 0 \vee 1]}$ as a switch before each algorithm, ensuring that only one of them operates at a time.

Two problems need to be solved here - the first is defining $\text{XYSwitch}$, and the second is providing an initial value for $\text{LB-Alg}$, since the conditions for its activation differ slightly from the case of the zero intersection point.

The new check should determine a specific range of values for the position of the vertical line. If the line is within this range, it should return 1; otherwise, it should return 0. In *Figure 141*, we can see that the line is positioned slightly after the intersection of the spiral with the $x$-axis, where $\text{LB-Alg}$ grounds the radius vector for the corresponding coil, in this case, $n=1$. The intersection point of the spiral with the $x$-axis will serve as the lower boundary. As the upper boundary, we will take the length of the radius vector at the point where the spiral intersects the next coordinate axis, i.e., the $y$-axis.

In [9]:
load_image('fig143')

If we look at the values of the lower and upper boundaries from *Figures 141 and 143*, we will see that $x_l$ is positioned between them - $-0.63617... < -0.437 < -0.42403...$ ($\text{Spiral_y} < \text{x_line} < \text{Spiral_x}$ on the images).

Here is how the interval looks.

In [10]:
load_image('fig144')

The check must determine whether the line is within this interval and on the same side of the spiral turn. The latter is necessary because the line could be positioned on the other side of the $x$-axis, in which case the check would also return 1.

Therefore, $\text{XYSwitch}$ represents the product of two checks.

$$XYSwitch_{(x_l, x_s, y_s)}^{[ 0 \vee 1]} = (\text{Xbetween})(\text{Xsigns}), \quad\quad\quad\quad(2.2.2.2)$$

where $\text{Xbetween}$ checks the interval, and $\text{Xsigns}$ checks the signs of $x_l$ and $x_s$.

We have already performed an interval check once before - the $\text{KL}$-check.

$$ KL_{(k, x_l)}^{[0 \vee  1]} = \lfloor \frac{1+\frac{x_l(k-1)(k-3)}{|E_{(x_l(k-1)(k-3))}|}}{2} \rfloor.   \quad\quad\quad\quad(2.2.1.7)$$

The $\text{Xbetween}$ check examines the relationship between three lengths and determines whether one of them - $x_l$ is positioned between the other two, namely between $x_s$ and $y_s$. For this purpose, we rewrite the $\text{KL}$ check as follows:

$$ Xbetween_{(x_l, x_s, y_s)}^{[0 \vee  1]} = \lfloor \frac{1-\frac{(|x_l| - |x_s|)(|y_s|-|x_l|)}
{|E_{((|x_l| - |x_s|)(|y_s|-|x_l|))}|}}{2} \rfloor.   \quad\quad\quad\quad(2.2.2.3)$$

Now we need to define the two radius vectors $x_s$ and $y_s$. Since we are performing a check related to a specific initial radius vector (in this case, from *Figure 141*, this is the first initial radius vector or $y_1$), we will use it as a reference point to obtain the lengths of $x_s$ and $y_s$. These two radius vectors can be obtained by rotating the initial vector by 90 and 180 degrees forward in time, respectively. Purely schematically:

$$ x_s = \vec{y_1} + 90^{\circ}, \quad\quad\quad\quad(2.2.2.3)$$

$$ y_s = \vec{y_1} + 180^{\circ}. \quad\quad\quad\quad(2.2.2.4)$$


Let’s recall how we obtained the initial radius vectors.

$$t_n = \frac{\pi\left(
Greather_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^+ +
Less_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^- +
(n-1)\right)}{2|E_{(\omega)}^{[1 \vee  \omega]}|}. \quad\quad\quad\quad(2.1.31)$$

Therefore, to obtain the length of the radius vector $x_s$, we need to add $\frac{\pi}{2}$ radians to the initial angle.

$$t_{n + 1/2} = \frac{\pi\left(
Greather_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^+ +
Less_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^- +
(n-1) + \frac{\pi}{2} \right)}{2|E_{(\omega)}^{[1 \vee  \omega]}|}. \quad\quad\quad\quad(2.2.2.5)$$

For adding an angle of $\pi$ radians for the next initial radius vector, we will simply take the next initial moment, i.e., $n+1$.

$$t_{n + 1} = \frac{\pi\left(
Greather_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^+ +
Less_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^- +
n \right)}{2|E_{(\omega)}^{[1 \vee  \omega]}|}. \quad\quad\quad\quad(2.2.2.6)$$

Now, for purely pragmatic reasons, we will change the values we compare in this check, and instead of comparing the lengths $x_l$, $x_s$ and $y_s$ we will compare their equivalent values in relation to $t$. To achieve this, we will convert the position of the vertical line $x_l$ into time, and thus for $t_{x_l}$ we have:

$$ t_{x_l} = \frac{|x_l|}{E_{(v)}}. \quad\quad\quad\quad(2.2.2.7)$$

Fewer operations are required to convert the length $|x_l|$ at moment $t_{x_l}$ than to convert the moments $t_{n + 1/2}$ and $t_{n + 1}$into the lengths of the radius vectors $x_s$ and $y_s$ using the formulas for the components $x(t_{n + 1/2}) = vt_{n + 1/2}cos(k \frac{\pi}{2} + \omega t_{n + 1/2})$ and $y(t_{n + 1}) = vt_{n + 1}sin(k \frac{\pi}{2} + \omega t_{n + 1})$. Therefore, for convenience, we adopt the notation $t_{n + 1/2}$ for the moment of the radius vector $x_s$, even though $n$ takes integer values.

Substituting these values into 2.2.2.3, we obtain:

$$ Xbetween_{(x_l, v, \omega, k)}^{[0 \vee  1]} = \lfloor \frac{1+\frac{(\frac{|x_l|}
{E_{(v)}} - t_{n + 1/2})(t_{n + 1}-\frac{|x_l|}{E_{(v)}})}
{|E_{\left((\frac{|x_l|}{E_{(v)}} - t_{n + 1/2})(t_{n + 1}-\frac{|x_l|}{E_{(v)}})\right)}|} }{2} \rfloor.   \quad\quad\quad\quad(2.2.2.8)$$

*Note:* Unlike $x_l$, the moments $t_{n + 1/2}$ and $t_{n + 1}$ are always positive since the domain of definition of $t$ is $t \ge 0$, so there is no need to take their absolute values. We also change the arguments passed to $\text{Xbetween}$, replacing $x_s$ and $y_s$ with $v$, $\omega$ and $k$, as they are the necessary parameters for calculating $t_{n + 1/2}$  and $t_{n + 1}$ .


With this, the $\text{Xbetween}$ check is completed, and what remains is to define $\text{XSigns}$. Since it needs to determine whether $x_l$ and $x_{t+1/2}$ are on the same side of the $x$-axis (see *Figure 141*), it will check the sign of the product between them.

$$ \text{XSigns}_{(x_l, v, \omega, k)}^{[0 \vee  1]} = \lfloor \frac{1 + \frac{x_l x(t_n +1/2)}{|E_{(x_l x(t_n +1/2))}|}}
{2} \rfloor, \quad\quad\quad\quad(2.2.2.10)$$

where

$$ x(t_n +1/2) = vt_{n + 1/2}cos(k \frac{\pi}{2} + \omega t_{n + 1/2}). \quad\quad\quad\quad(2.2.2.11)$$

The final step is to substitute the checks $\text{Xbetween}$ and $\text{XSigns}$ into the formula for $\text{XYSwitch}$.

$$ XYSwitch_{(x_l, v, \omega, k)}^{[ 0 \vee 1]} = 
\left(
\lfloor \frac{1+\frac{(\frac{|x_l|}
{E_{(v)}} - t_{n + 1/2})(t_{n + 1}-\frac{|x_l|}{E_{(v)}})}
{|E_{\left( (\frac{|x_l|}{E_{(v)}} - t_{n + 1/2})(t_{n + 1}-\frac{|x_l|}{E_{(v)}})\right)}|} }{2}
\rfloor
\right)
\left(
\lfloor \frac{1 + \frac{x_l x(t_n +1/2)}{|E_{(x_l x(t_n +1/2))}|}}
{2} \rfloor
\right)
\quad\quad\quad\quad(2.2.2.12)$$

The opposite check is therefore:

$$ \overline {XYSwitch_{(x_l, v, \omega, k)}^{[ 0 \vee 1]}} = 1 - XYSwitch_{(x_l, v, \omega, k)}^{[ 0 \vee 1]} \quad\quad\quad\quad(2.2.2.13)$$

After defining $\text{XYSwitch}$, the remaining task is to resolve the issue with the input value of $\text{LB-Alg}$. Let's recall its formula:

$$ t_{[0;m]} = \frac{|x_l|}
{E_{(v)}\cos(\arctan(|\frac{y_s(t_{[0;m-1]})}{E_{(x_s(t_{[0;m-1]}))}}|))}, \quad\quad\quad\quad (2.2.19)$$

where the input value of the algorithm was determined based on the position $x_l$ of the line.

$$ t_{[0;0]} = \frac{|x_l|}{E_{(v)}}. \quad\quad\quad\quad (2.2.14)$$

If we use this formula, it will lead to calculating a dynamic input value as long as $x_l$ is within the length interval between $y_s$ and $x_s$. In the images below, it can be seen that the input value has different lengths, and in *Figure 146*, it moves behind $\text{PMD}$.

In [12]:
display(load_images([('fig145', 'left'), ('fig146', 'right')]))

It is unlikely, but under a certain configuration of the spiral and line parameters, this could still lead to calculating an incorrect 'intersection point.' To avoid such a possible scenario, we will use a different length as the input parameter. This length will remain static for any position of the line. It will be the same radius vector that we used as the upper boundary of the interval in $\text{XYSwitch}$.

In [14]:
display(load_images([('fig147', 'left'), ('fig148', 'right')]))

As can be seen, shifting the line to the same position as in *Figure 146* does not change the initial radius vector.

All this means that we need to redefine $\text{LB-Alg}$ for each $n$-th initial radius vector. Then we have:

$$ t_{[n;m]} = \frac{|x_l|}
{E_{(v)}\cos(\arctan(|\frac{y_s(t_{[n;m-1]})}{E_{(x_s(t_{[n;m-1]}))}}|))}, \quad\quad\quad\quad (2.2.2.14)$$

where the initial moment $t_{[n;0]} = t_{n+1/2}$ from 2.2.2.5.

$$t_{[n;0]} = \frac{\pi\left(
Greather_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^+ +
Less_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^- +
(n-1) + \frac{\pi}{2} \right)}{2|E_{(\omega)}^{[1 \vee  \omega]}|}. \quad\quad\quad\quad(2.2.2.15)$$

However, the actual initial value is not determined based on the length of the radius vector $x_s$, but rather on the position $x_l$. The value of the $y$-component of this radius vector is 0 (*Figure 147*), which means that the multiplier $\cos(\arctan(|\frac{y_s(t_{[n;m-1]})}{E_{(x_s(t_{[n;m-1]}))}}|))$ in the denominator in 2.2.2.14 equals 1. Then:

$$ t_{[n;0]} = \frac{|x_l|}
{E_{(v)}}. \quad\quad\quad\quad (2.2.2.16)$$

This extension of the definition of $\text{LB-Alg}$ requires giving it a different name when it is executed in cases for $n \ge 1$. We will call it $\text{NLB-Alg}$, and thus the formula for the two-dimensional sequence of intersection points will have the following schematic form:

$$\lim_{n, m \to \infty} \left\{ t_{[n; m]} \right\} =
\text{ISSCDD}_{(x'_s(t_{[n;0]}), x'_s(t_{[n;m]}))}^{[0 \vee 1]}
\left( \overline{NSwitch_{(n)}^{[0 \vee  1]}}
\text{XMD}_{(x_l, y_s(t_{[1; 0]}))}^{[0 \vee 1]}
\text{SCDD}_{(x'_s(0), x'_s(t_{[0;0]}))}^{[0 \vee 1]}\\
\left(
KWL_{(k, w, x_l)}^{[0 \vee  1]} +
KL_{(k, x_l)}^{[0 \vee  1]}
\right)
\text{LB-Alg}
+ NSwitch_{(n)}^{[0 \vee  1]}
\left( \overline{XYSwitch_{(x_l, v, \omega, k)}^{[ 0 \vee 1]}} \text{AB-Alg} +
XYSwitch_{(x_l, v, \omega, k)}^{[ 0 \vee 1]} \text{NLB-Alg}
\right)
\right), \quad n, m \ge 0.  \quad\quad\quad\quad (2.2.2.17)$$

Now, let's return to the scenario from *Figure 147* and see how the new check $\text{XYSwitch}$ and the algorithm $\text{NLB-Alg}$ work.

In [21]:
display(load_images([('fig149', 'left'), ('fig150', 'right')]))

In *Figure 149*, we see that $\text{XYSwitch}$ activates $\text{NLB-Alg}$, $\overline{\text{XYSwitch}}$ deactivates $\text{AB-Alg}$ and  moves the initial vector from the positive ordinate axis to the negative abscissa axis. In the other figure, we see that the radius vector in the first iteration ($m=1$) is longer than the initial vector and has the same length as $x_l$.

We will examine another scenario to ensure that all checks, including the new one, function correctly. 

In [22]:
display(load_images([('fig151', 'left'), ('fig152', 'right')]))

In *Figure 151*, we see that $\text{NLB-Alg}$ is activated for the $6$-th initial radius vector, and it is correctly rotated by 90 degrees to the abscissa axis. In the next figure, we see that the line is shifted slightly to the right, where it does not intersect the spiral loop. Here, $\text{ISSCDD}$ is triggered and disables $\text{NLB-Alg}$ (as well as all other algorithms) already in the first iteration.

Now that we have corrected this issue in the algorithms, the final problem remains. It can be seen in *Figure 152*. We will display it in a larger size.

In [23]:
load_image('fig152')

The problem lies in the points aligned along the $x$-axis at the locations where the spiral curve intersects it. For this specific parameters of the spiral and the position of the line, these are the even initial vectors, or those on the positive side of the $y$-axis. Let's verify that this is indeed the case.

In [25]:
display(load_images([('fig153', 'left'), ('fig154', 'right')]))

The even initial radius vector $n=2$ 'lands' on the $x$-axis at the $11$-th iteration.

In [26]:
display(load_images([('fig155', 'left'), ('fig156', 'right')]))

In contrast to the even radius vector, the odd one $n=3$ passes through $\text{PMD}$, and $\text{ISSCDD}$ resets $\text{NLB-Alg}$ as early as the second iteration. (The green radius vector in *Figure 156* is the radius vector from the previous iteration).


To get a clearer idea of what the new check should do, let's look at the image below. It shows a spiral and a vertical line that does not intersect the first three coils. On these coils, the algorithm incorrectly positions intersection points on the x-axis. Therefore, the new check must disable the algorithm only for the first three coils, but not for all the others.

In [28]:
load_image('fig157')

This can be achieved if the check verifies whether the line is located behind a specific coil. If it is behind it, it will return 0; if it is in front of it, it will return 1. More specifically, the check will be performed in relation to the intersection point of the spiral with the $x$-axis.

The new check will be called $\text{ABSwitch}$, as its role will be to decide whether to activate $\text{AB-Alg}$, i.e., for $n \ge 1$. $\text{LB-Alg}$ is not involved in these calculations because we have already defined other checks that determine under what conditions it should be activated.

$\text{ABSwitch}$, like $\text{XYSwitch}$, will compare lengths but as we did with $\text{XYSwtich}$, we will transform the lengths into time.

$$\text{ABSwitch}_{(x_l, v, \omega, k)}^{[ 0 \vee 1]} = t_{n \pm 1/2} + C + \frac{|x_l|}{E_{(v)}}.\quad\quad\quad\quad (2.2.2.18)$$

Two things need to be explained in this formula. One is the plus-minus sign in $t_{n \pm 1/2}$ and the constant $C$. The double sign for addition needs to be included because, depending on the number $n$, either the time $\frac{\pi}{2E_{(w)}}$ needs to be added or the same time needs to be subtracted from the initial vector. The images below illustrate this.

In [29]:
display(load_images([('fig158', 'left'), ('fig159', 'right')]))

Due to the position of the line on the negative abscissa, the specified time must be added to the time of the initial vector $n=5$, while for the next initial vector $n=6$, the same time must be subtracted in order to find the length of the radius vector lying on the abscissa, relative to which we will compare the position of the line $x_l$. Which of the two operations should be performed will be determined through an algebraic check. Such a check has already been defined in the angular algorithm. This is the sign check from 2.1.3.10.


$$S_{(-\omega x_l y_s(t_{[n; m-1]}))} = \frac{-\omega x_l y_s(t_{[n; m-1]})}{|E_{(-\omega x_l y_s(t_{[n; m-1]}))}|}. \quad\quad\quad\quad(2.1.3.10)$$

We will slightly modify the formula by replacing $m-1$ with $0$. This gives us:

$$S_{(-\omega x_l y_s(t_{[n; 0]}))} = \frac{-\omega x_l y_s(t_{[n; 0]})}{|E_{(-\omega x_l y_s(t_{[n; 0]}))}|}. \quad\quad\quad\quad(2.2.2.19)$$

For $n=5$ from *Figure 158*, we add accordingly, while for $n=6$ from the next figure, we subtract. However, we will perform this check outside the time formula to avoid unnecessary modifications to it. Let us recall once again:

$$t_n = \frac{\pi\left(
Greather_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^+ +
Less_{(\omega)}^{[0 \vee 1]}\Delta \theta(k)^- +
(n-1)\right)}{2|E_{(\omega)}^{[1 \vee  \omega]}|}. \quad\quad\quad\quad(2.1.31)$$

With the addition of the check, the formula 2.2.2.18 takes the form:

$$\text{ABSwitch}_{(x_l, v, \omega, k)}^{[ 0 \vee 1]} = t_{[n; 0]} +
S_{(-\omega x_l y_s(t_{[n; 0]}))} 
+ C + \frac{|x_l|}{E_{(v)}}.\quad\quad\quad\quad (2.2.2.20)$$